<a href="https://colab.research.google.com/github/nunzio998/REC-Energy-Management/blob/main/NLP_Task/NLP_TASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT

In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import re

LOAD DATASET

In [2]:
%%capture
!pip install datasets

In [3]:
# Load del dataset
from datasets import load_dataset

ds = load_dataset("big_patent")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for big_patent contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/big_patent
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/75 [00:00<?, ?it/s]

VISUALIZZAZIONE DATASET

In [4]:
# Visualizzazione dataset
print(ds)
# print(ds["test"]["description"][1])
# print("\n" + ds["test"]["abstract"][1])

DatasetDict({
    train: Dataset({
        features: ['description', 'abstract'],
        num_rows: 1207222
    })
    validation: Dataset({
        features: ['description', 'abstract'],
        num_rows: 67068
    })
    test: Dataset({
        features: ['description', 'abstract'],
        num_rows: 67072
    })
})


TRAIN/TEST SPLIT DATASET

In [ ]:
# Split del dataset

x_train_text = ds['train']['description']
y_train = ds['train']['abstract']

In [5]:
x_validation_text = ds['validation']['description']
y_validation = ds['validation']['abstract']

In [ ]:
x_test_text = ds['test']['description']
y_test = ds['test']['abstract']
#print("-----------\n" + x_test_text[1])

# PREPROCESSING DATI

TEXT CLEANING


*   HTML tags
*   punctuations by negating chars, spaces, and digits
*   consecutive spaces





In [6]:
# Trattamento dei dati tramite regex

def noise_remover(text):
    text2 = re.sub('<[^>]*>', '', text)  # remove html tags
    text2 = re.sub('[^\w\s\d]', ' ', text2)  # remove punctuations by negating chars, spaces, and digits
    text2 = re.sub('\s+', ' ', text2)  # remove consecutive spaces
    #print(text2)
    return text2


# Rimozione rumore
# Utilizzo tqdm per monitorare l'avanzamento

#x_train_text_processed = []
#for el in x_train_text:
  #x_train_text_processed.append(noise_remover(el))

x_validation_text_processed = []
for el in x_validation_text:
  x_validation_text_processed.append(noise_remover(el))

#x_test_text_processed = []
#for el in x_test_text:
  #x_test_text_processed.append(noise_remover(el))


#print("-----------\n" + x_test_text[1])

TOKENIZZAZIONE

In [7]:
%%capture
!pip install nltk

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
# Download the Punkt tokenizer model
nltk.download('punkt')

#x_train_text_tokenized = []
#for el in x_train_text_processed:
  #x_train_text_tokenized.append(word_tokenize(el))

x_validation_text_tokenized = []
for el in x_validation_text_processed:
  x_validation_text_tokenized.append(word_tokenize(el))

#x_test_text_tokenized = []
#for el in x_test_text_processed:
  #x_test_text_tokenized.append(word_tokenize(el))

print(x_validation_text_processed[1] + "\n---------------------------------------------------------\n")
print(x_validation_text_tokenized[1]) # print the list of tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TEXT NORMALIZATION:
Lowercasing, Stemming, Lemmatization and stop words removal

In [ ]:
#STOP WORDS REMOVAL

import nltk
nltk.download('stopwords') # Download the stopwords module

stopwords = nltk.corpus.stopwords.words(
    "english"
) # Get the list of stop words for English

len(stopwords), stopwords[:10] # print stopwords



In [ ]:
# STEMMING

from nltk import LancasterStemmer

# create an object of class LancasterStemmer
lancaster = LancasterStemmer()

#x_train_text_stemmed = []
#for el in x_train_text_processed:
  #stemmed_tokens = [
            #lancaster.stem(token.lower())  # Stemming
            #for token in word_tokenize(el)  # Tokenize
            #if token.lower() not in stopwords  # Stopwords removals
        #]
  #stemmed_train_text = " ".join(stemmed_tokens)
  #x_train_text_stemmed.append(stemmed_train_text)

x_validation_text_stemmed = []
for el in x_validation_text_processed:
  stemmed_tokens = [
            lancaster.stem(token.lower())  # Stemming
            for token in word_tokenize(el)  # Tokenize
            if token.lower() not in stopwords  # Stopwords removals
        ]
  x_validation_text_stemmed.append(stemmed_tokens)
  print(x_validation_text_tokenized[1] + "\n---------------------------------------------------------\n")
  print(x_validation_text_stemmed[1])


#x_test_text_stemmed = []
#for el in x_test_text_processed:
  #stemmed_tokens = [
            #lancaster.stem(token.lower())  # Stemming
            #for token in word_tokenize(el)  # Tokenize
            #if token.lower() not in stopwords  # Stopwords removals
        #]
  #stemmed_test_text = " ".join(stemmed_tokens)
  #x_test_text_stemmed.append(stemmed_test_text)

In [ ]:
# LEMMATIZATION

from nltk import WordNetLemmatizer
nltk.download('wordnet') # Download the wordnet resource

# create an object of class LancasterStemmer
wnet_lemma = WordNetLemmatizer()

#x_train_text_lemmatized = []
#for el in x_train_text_processed:
  #lemma_tokens = [
            #wnet_lemma.lemmatize(token.lower())  # Lemmatization
            #for token in word_tokenize(el)  # Tokenize
            #if token.lower() not in stopwords  # Stopwords removals
        #]
  #lemmatized_train_text = " ".join(lemmatized_tokens)
  #x_train_text_lemmatized.append(lemmatized_train_text)

x_validation_text_lemmatized = []
for el in x_validation_text_processed:
  lemma_tokens = [
            wnet_lemma.lemmatize(token.lower())  # Lemmatization
            for token in word_tokenize(el)  # Tokenize
            if token.lower() not in stopwords  # Stopwords removals
        ]
  x_validation_text_lemmatized.append(lemma_tokens)
  print(x_validation_text_tokenized[1] + "\n---------------------------------------------------------\n")
  print(x_validation_text_lemmatized[1])

#x_test_text_lemmatized = []
#for el in x_test_text_processed:
  #lemma_tokens = [
            #wnet_lemma.lemmatize(token.lower())  # Lemmatization
            #for token in word_tokenize(el)  # Tokenize
            #if token.lower() not in stopwords  # Stopwords removals
        #]
  #lemmatized_test_text = " ".join(lemmatized_tokens)
  #x_test_text_lemmatized.append(lemmatized_test_text)